In [ ]:
import json
import time
import pandas as pd
from py2neo import Graph, Node, Relationship
#from py2neo.Graph import database 

# Need to get authentication working, currently NEO4J_AUTH=none
graph = Graph("bolt://neo4j:7687")
# graph = Graph('bolt://localhost:7687', bolt=True)

#graph.delete_all()

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

### Get percentile cutoffs of PageRank score

In [ ]:
query = """
MATCH (n:Quanta)
WHERE n.year>1990 AND (n.venue="Nature" OR n.venue="Science")
WITH SIZE((n)<-[:CITES]-(:Quanta)) as citations, n.pageRank_2018 as pagerank
RETURN 
percentileCont(citations, 0.05),
percentileCont(pagerank, 0.95)
"""
# print(query)
query_start_time = time.time()
df = graph.run(query).to_data_frame()
print("Done in {:.2f} minutes.".format((time.time()-query_start_time)/60))

high_pagerank_threshold = 1.4731184999999998

In [ ]:
df

In [ ]:
query = """
MATCH (n:Quanta) 
WHERE (n.year > 1990) AND (n.venue="Nature" OR n.venue="Science")
WITH 
    100 as citation_threshold, 
    percentileCont(n.pageRank_2018, 0.90) as pagerank_threshold
MATCH (q:Quanta) 
WHERE 
    (q.year > 1990) AND (q.venue="Nature" OR q.venue="Science") AND
    (q.pageRank_2018 >= pagerank_threshold) AND 
    (q.n_citation <= citation_threshold) AND
    EXISTS(q.fos)
RETURN 
q.title as title, 
q.year as year, 
q.pageRank_2018 as pagerank, 
q.n_citation as citations
ORDER BY pagerank DESC
"""
# print(query)
query_start_time = time.time()
df = graph.run(query).to_data_frame()
print("Done in {:.2f} minutes.".format((time.time()-query_start_time)/60))



In [ ]:
df